In [45]:
import pandas as pd
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [46]:
train_input=pd.read_csv("CRedit_Risk_Train_data.csv")
test_input=pd.read_csv("Credit_Risk_Validate_data.csv")

In [47]:
print(train_input.columns)
print(test_input.columns)

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')
Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'outcome'],
      dtype='object')


In [48]:
#The lst column has a differrent bname in both the columns
#lets make the names same first and then merge both of them
#so that we will fill thw missing v values simultaneosuly
test_input.rename(columns={"outcome": "Loan_Status"},inplace=True)


In [49]:
data_all=pd.concat([train_input,test_input],axis=0)
data_all.shape

(981, 13)

In [50]:
data_all.reset_index(inplace=True,drop=True)
#reset index else merging wioll havw issues
print(data_all.tail())

      Loan_ID Gender Married Dependents     Education Self_Employed  \
976  LP002971   Male     Yes         3+  Not Graduate           Yes   
977  LP002975   Male     Yes          0      Graduate            No   
978  LP002980   Male      No          0      Graduate            No   
979  LP002986   Male     Yes          0      Graduate            No   
980  LP002989   Male      No          0      Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
976             4009             1777.0       113.0             360.0   
977             4158              709.0       115.0             360.0   
978             3250             1993.0       126.0             360.0   
979             5000             2393.0       158.0             360.0   
980             9200                0.0        98.0             180.0   

     Credit_History Property_Area Loan_Status  
976             1.0         Urban           Y  
977             1.0         Urban     

In [51]:
data_all.isnull().sum()#gives sum of missing valeus of the columns

Loan_ID               0
Gender               24
Married               3
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [52]:
data_all.shape # read the decription o each colum from teh word document

(981, 13)

In [53]:
#before buildinng the modle first lets clear ooue thwe misisng values


In [54]:
Counter(data_all["Gender"])

Counter({'Male': 775, 'Female': 182, nan: 24})

In [55]:
#lets fill them by MAle
#these rows are of null gender
#lest fill the null gemnder with male
gender_null=data_all[data_all["Gender"].isnull()].index.tolist()
print(gender_null)

[23, 126, 171, 188, 314, 334, 460, 467, 477, 507, 576, 588, 592, 636, 665, 720, 752, 823, 845, 859, 893, 910, 917, 932]


In [56]:
gender_null_M=gender_null[:12]
gender_null_F=gender_null[12:]
data_all["Gender"].iloc[gender_null_M]="Male"
data_all["Gender"].iloc[gender_null_F]="Female

SyntaxError: EOL while scanning string literal (<ipython-input-56-43c4fabaa647>, line 4)

In [57]:
data_all["Gender"].iloc[gender_null]="Male"


In [58]:
#check if failed
print(sum(data_all["Gender"].isnull()))#okay done
Counter(data_all["Gender"])

0


Counter({'Male': 799, 'Female': 182})

In [59]:
#lets filled married now
print(Counter(data_all["Married"]))#almost all are marrieed

Counter({'Yes': 631, 'No': 347, nan: 3})


In [60]:
married_null=data_all[data_all["Married"].isnull()].index.tolist()
married_null

[104, 228, 435]

In [61]:
data_all["Married"].iloc[married_null]="Yes"


In [62]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [63]:
Counter(data_all["Dependents"])

Counter({'0': 545, '1': 160, '2': 160, '3+': 91, nan: 25})

In [64]:
#lets see the dpendednce wrrt marrinage
pd.crosstab(data_all["Married"],data_all["Dependents"].isnull())


Dependents,False,True
Married,,
No,338,9
Yes,618,16


In [65]:
pd.crosstab(data_all["Dependents"],data_all["Married"])

Married,No,Yes
Dependents,,
0,276,269
1,36,124
2,14,146
3+,12,79


In [66]:
#for the batchlors lets fill the msiing dependts with 0
#lets find the index of ala rows with dependent missing and married NO
bachelor_nulldependent=data_all[(data_all["Married"]=="No")&(data_all["Dependents"].isnull())].index.tolist()
print(bachelor_nulldependent)


[293, 332, 355, 597, 684, 752, 879, 916, 926]


In [67]:
data_all["Dependents"].iloc[bachelor_nulldependent]="0"

In [68]:
Counter(data_all["Dependents"])

Counter({'0': 554, '1': 160, '2': 160, '3+': 91, nan: 16})

In [69]:
pd.crosstab(data_all["Gender"],data_all["Dependents"])

Dependents,0,1,2,3+
Gender,,,,
Female,127,32,13,9
Male,427,128,147,82


In [70]:
pd.crosstab(data_all["Gender"],data_all["Dependents"].isnull())

Dependents,False,True
Gender,,
Female,181,1
Male,784,15


In [71]:
pd.crosstab((data_all["Gender"]=="Male")&(data_all["Married"]=="Yes"),data_all["Dependents"])

Dependents,0,1,2,3+
row_0,,,,
False,318,48,23,15
True,236,112,137,76


In [72]:
#lest fill the dependnts with 1
data_all["Dependents"].iloc[data_all[data_all["Dependents"].isnull()].index.tolist()]="1"

In [73]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [74]:
Counter(data_all["Self_Employed"])

Counter({'No': 807, 'Yes': 119, nan: 55})

In [75]:
self_emp_null=data_all[data_all["Self_Employed"].isnull()].index.tolist()

In [76]:
##fill missingself employed with no
data_all["Self_Employed"].iloc[self_emp_null]="No"

In [77]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [82]:
# To check if any row with both LoanAmount and Loan_Amount_Term as NAN
pd.crosstab(data_all['LoanAmount'].isnull(),
            data_all['Loan_Amount_Term'].isnull())

Loan_Amount_Term,False,True
LoanAmount,,
False,934,20
True,27,0


In [83]:
pd.crosstab(data_all['LoanAmount'].isnull(),data_all['Loan_Amount_Term'])

Loan_Amount_Term,6.0,12.0,36.0,60.0,84.0,120.0,180.0,240.0,300.0,350.0,360.0,480.0
LoanAmount,,,,,,,,,,,,
False,1,2,3,3,7,4,64,7,20,1,800,22
True,0,0,0,0,0,0,2,1,0,0,23,1


In [84]:
data_all.groupby(data_all['Loan_Amount_Term'])['LoanAmount'].mean()

Loan_Amount_Term
6.0       95.000000
12.0     185.500000
36.0     117.666667
60.0     139.666667
84.0     121.142857
120.0     36.750000
180.0    131.125000
240.0    128.857143
300.0    166.250000
350.0    133.000000
360.0    144.420000
480.0    137.181818
Name: LoanAmount, dtype: float64

In [85]:
#lets fill the missing values in LoanAmount 
#with the mean of the respective Loan_Term
#we see that 180 & 240 has the almost same Loan amount 128-131
#& 360 has high i.e 144
#so lets fill only 360 by 144 
#and all remaining by 130
data_all['LoanAmount'][(data_all['LoanAmount'].isnull()) 
                       & (data_all['Loan_Amount_Term']==360)]=144
data_all['LoanAmount'][(data_all['LoanAmount'].isnull()) 
                       & (data_all['Loan_Amount_Term']==480)]=137

In [86]:
data_all['LoanAmount'][(data_all['LoanAmount'].isnull())]=130

In [87]:
#lets fill Loan Amount Term
(data_all['Loan_Amount_Term']).value_counts()

360.0    823
180.0     66
480.0     23
300.0     20
240.0      8
84.0       7
120.0      4
36.0       3
60.0       3
12.0       2
350.0      1
6.0        1
Name: Loan_Amount_Term, dtype: int64

In [88]:
# lets fill loadn tenture by the mode i.e 360
data_all["Loan_Amount_Term"][data_all["Loan_Amount_Term"].isnull()]=360

In [89]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [91]:
data_all["Credit_History"].value_counts()

1.0    754
0.0    148
Name: Credit_History, dtype: int64

In [93]:
pd.crosstab(data_all["Gender"],data_all["Credit_History"])
#GEnder makes no differnevce

Credit_History,0.0,1.0
Gender,,
Female,30,135
Male,118,619


In [94]:
pd.crosstab(data_all["Education"],data_all["Credit_History"])
#education makes no differrence



Credit_History,0.0,1.0
Education,,
Graduate,106,596
Not Graduate,42,158


In [95]:
pd.crosstab(data_all["Married"],data_all["Credit_History"])
#MArried makes no differrence


Credit_History,0.0,1.0
Married,,
No,56,263
Yes,92,491


In [97]:
data_all["Credit_History"][data_all["Credit_History"].isnull()]=1

In [98]:
data_all.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64